In [2]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent
# from attention_decoder import AttentionDecoder

In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [17]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_embedding2_dims = nb_embedding_dims
nb_sequence_length = 75

In [5]:
def twitter_tokenizer(textline):
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [6]:
def sequential_generator(filename, batch_size):
    
    file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
    shuffled_indexes = range(1, file_length + 1)
    # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
    index_position = 0
    
    batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
    batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
    # batch_features_idx = np.zeros((batch_size, nb_sequence_length))
    batch_labels = np.zeros((batch_size, 2))

    while True:
        # print(len(features))
        for i in range(batch_size):
            line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            batch_features_ft[i], batch_features_lg[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            index_position += 1
            if index_position == file_length:
                # shuffle indexes again
                shuffled_indexes = range(1, file_length + 1)
                # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
                index_position = 0
                break
        # yield [batch_features_ft, batch_features_lg], batch_labels
        yield [batch_features_ft], batch_labels

In [7]:
# word_splitter = re.compile("[\w+]|[\W+]", re.UNICODE)
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    # print(words)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_lg = np.zeros((nb_sequence_length, nb_embedding2_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        if w in word2Idx:
            wv = wordEmbeddings[word2Idx[w]]
            widx = word2Idx[w]
        else:
            wv = wordEmbeddings[word2Idx["UNKNOWN_TOKEN"]]
            widx = word2Idx["UNKNOWN_TOKEN"]
        features_lg[idx] = wv
        features_idx = widx
        
        idx = idx + 1
    return features_ft, features_ft

In [9]:
train_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_train.txt', encoding = "UTF-8")]
dev_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_dev.txt', encoding = "UTF-8")]

In [10]:
train_sentences = [x[0] for x in train_lines]
train_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in train_lines])
# train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

dev_sentences = [x[0] for x in dev_lines]
dev_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in dev_lines])
# dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

In [11]:
n_labels = 2

In [14]:
characters={}
for line in train_sentences:
    for char in line:
        characters[char] = True
for line in dev_sentences:
    for char in line:
        characters[char] = True
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)

In [18]:
nb_embedding2_dims = wordEmbeddings[1].shape[0]
# print(nb_embedding2_dims)
# print('für' in word2Idx)
# print(wordEmbeddings[word2Idx['für']])
# print('Ute' in word2Idx)
# print(wordEmbeddings[word2Idx['Ute']])


300
True
[ 0.048675  0.13134  -0.103707 -0.178012 -0.095551 -0.171765 -0.482186
 -0.595483  0.40201  -0.0764    0.030391  0.053942  0.125109 -0.100393
 -0.18863  -0.040022  0.124366  0.234751  0.170881 -0.14696  -0.096499
  0.076724  0.363462  0.15184   0.028145  0.108519 -0.384469  0.513485
 -0.080346  0.287282  0.129523 -0.205344 -0.189282  0.00303  -0.314163
 -0.253001 -0.191423 -0.3048    0.264441  0.04405  -0.715606  0.303819
  0.421922  0.370784  0.012607  0.262544 -0.100403  0.354938  0.012659
 -0.312128  0.202812  0.053679  0.115874  0.039126  0.058098  0.092229
  0.309017 -0.191012  0.020758 -0.226308 -0.363673  0.093612 -0.099952
  0.361019 -0.094479 -0.422336 -0.18642  -0.236536 -0.519167  0.159582
  0.338029 -0.18979   0.180626  0.125307 -0.662534  0.035188  0.484701
 -0.335092 -0.1776   -0.227792  0.038483 -0.3802   -0.534377 -0.782993
 -0.089097 -0.205562 -0.044407 -0.051924 -0.575704  0.018059 -0.053812
  0.283002  0.514965 -0.486068  0.055361 -0.054333 -0.020527 -0.4472

In [12]:
model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
lstm_block = LeakyReLU()(lstm_block)

filter_sizes = (3, 4, 5)
conv_blocks = []
for sz in filter_sizes:
    conv = Conv1D(
        filters = 200,
        kernel_size = sz,
        padding = 'valid',
        strides = 1
    )(lstm_block)
    conv = LeakyReLU()(conv)
    conv = GlobalMaxPooling1D()(conv)
    conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
# model_concatenated = Dropout(0.8)(model_concatenated)
model_concatenated = Dense(100)(model_concatenated)
model_concatenated = LeakyReLU()(model_concatenated)
model_output = Dense(n_labels, activation = "softmax")(model_concatenated)
model = Model(model_input_embedding, model_output)
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 75, 200)      320800      input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 75, 200)      0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 73, 200)      120200      leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
conv1d_2 (

In [13]:
samples_per_epoch = len(train_sentences)
epochs = 50
batch_size = 32
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
checkpoint = ModelCheckpoint('best_classification_model.h5', 
                             monitor='val_acc', 
                             verbose = 1, 
                             save_best_only = True, 
                             save_weights_only = True)

In [18]:
dev_batch_size = len(dev_sentences)
model.fit_generator(
    sequential_generator('/home/gwiedemann/notebooks/OffLang/sample_train.txt', batch_size), 
    steps_per_epoch=steps_per_epoch, epochs=epochs,
    validation_data = sequential_generator('/home/gwiedemann/notebooks/OffLang/sample_dev.txt', dev_batch_size),
    validation_steps = math.ceil(len(dev_sentences) / dev_batch_size),
    callbacks = [checkpoint]
)
model.reset_states()

Epoch 1/50


NameError: name 'word2Idx' is not defined

In [206]:
model.load_weights('best_classification_model.h5')
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
testset_features_e1 = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
testset_features_e2 = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding2_dims))   
for i in range(len(dev_sentences)):
    testset_features_e1[i], testset_features_e2[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
results = model.predict(testset_features_e1)

In [207]:
idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
predLabels = results.argmax(axis=-1)
devLabels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
# print(idx2Label)
# print(predLabels)
# print(devLabels)
f1 = f1_score(devLabels, predLabels, average='binary', pos_label=1)
r = recall_score(devLabels, predLabels, average='binary', pos_label=1)
p = precision_score(devLabels, predLabels, average='binary', pos_label=1)
a = accuracy_score(devLabels, predLabels)
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))

Test-Data: Prec: 0.738, Rec: 0.685, F1: 0.711, Acc: 0.816
